In [1]:
import sys 
sys.path.append('../') 

In [2]:
sample_file = "/Users/amycweng/DH/EEPS/A41135_encoded.csv"
citations_file = "/Users/amycweng/DH/SERMONS_APP/db/data/Interregnum/A4_citations.csv"
qp_file = "/Users/amycweng/DH/SERMONS_APP/db/data/Interregnum/paraphrases.csv"

In [3]:
import pandas as pd 
encoding = pd.read_csv(sample_file)
citations = pd.read_csv(citations_file,header=None)
citations = citations[citations[0] == 'A41135']
qp = pd.read_csv(qp_file,header=None)
qp = qp[qp[0] == 'A41135']

In [4]:
encoding = encoding.to_dict(orient='records')

In [5]:
sindices = citations[1].tolist()
s_cited = citations[6].tolist()
c_dict = {}
c_endings = {}
for idx,sidx in enumerate(sindices): 
    if sidx not in c_dict: c_dict[sidx] = ([],[]) 
    c = s_cited[idx].split(' ')
    if len(c[0]) == 1: 
        start = c[1]
    else: start = c[0]
    end = c[-1]
    c_dict[sidx][0].append(start)
    c_dict[sidx][1].append(end)

In [6]:
qp_indices = qp[1].tolist()

In [7]:
encoding[0].keys()

dict_keys(['token', 'pos', 'regular', 'lemma', 'note_tag', 'it_tag', 'sent_idx', 'section_idx', 'paragraph_idx', 'section_name'])

In [ ]:
citation_enc = []
in_cited = False 

def skip(sidx): 
    # get the preceding and succeeding segments of those with citations or QP 
    if sidx-1 in c_dict: return False
    if sidx+1 in c_dict: return False 
    if sidx+1 in qp_indices: return False 
    if sidx-1 in qp_indices: return False
    if sidx in qp_indices: return False 
    return True 

for e in encoding: 
    sidx = e['sent_idx']
    if isinstance(e['token'],float): 
        continue 
    if sidx not in c_dict: 
        e['cite_tag'] = 'O'
        if skip(sidx): 
            continue  
    elif e['token'].strip(".,") in c_dict[sidx][0]: 
        e['cite_tag'] = 'B-S' 
        in_cited = True
    elif in_cited:
        if e['token'].strip(".,") in c_dict[sidx][1]: 
            in_cited = False 
        e['cite_tag'] = 'I-S'
    else:  
        e['cite_tag'] = 'O' 
    
    if sidx in qp_indices: 
        e['qp_tag'] = True
    else: 
        e['qp_tag'] = False 
    citation_enc.append(e) 

In [18]:
df = pd.DataFrame(citation_enc)
df[df['cite_tag'] == 'B-S']

,token,pos,regular,lemma,note_tag,it_tag,sent_idx,section_idx,paragraph_idx,section_name,cite_tag,qp_tag
10,COLOS,np1,COLOS,COLOS,0,0,1,3,0,treatise,B-S,False
474,Rom.,np1,Rom.,Rom.,0,0,47,3,11,treatise,B-S,True
568,Psal.,np1,Psal.,Psal.,0,0,50,3,11,treatise,B-S,True
606,Psal.,np1,Psal.,Psal.,0,1,54,3,11,treatise,B-S,True
617,Rom.,np1,Rom.,Rom.,0,1,55,3,11,treatise,B-S,False
...,...,...,...,...,...,...,...,...,...,...,...,...
42856,Job,n1,Job,job,0,1,4629,3,226,treatise,B-S,False
43153,Pro.,np1,Pro.,Pro.,0,1,4670,3,227,treatise,B-S,False
43461,Pet.,np1,Pet.,Pet.,0,1,4713,3,228,treatise,B-S,True
43539,Pro.,np1,Pro.,Pro.,0,1,4724,3,228,treatise,B-S,False


In [154]:
df.to_csv("/Users/amycweng/DH/EEPS/A41135_citations.csv",index=False)